In [1]:
import keras
import tensorflow as tf

In [2]:
# 對於訓練資料中的5種類別的花，做圖像辨識，讓模型能準確分辨出測試資料每張照片分別是哪種種類的花
# 第一步先載入需要用到的模組
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,RMSprop
from keras import backend as K
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import pandas as pd
import os

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
# 首先要先設定CNN相關參數
# 資料路徑
DATASET_PATH  = './train'

# 影像大小
IMAGE_SIZE = (224, 224)

BATCH_SIZE = 16    # 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
NUM_CLASSES = 5   # 影像類別數
FREEZE_LAYERS = 5 # 凍結網路層數
NUM_EPOCHS = 200   # Epoch 數
WEIGHTS_FINAL = 'model-resnet50-final.h5'# 模型輸出儲存的檔案
WEIGHTS_LAST = 'model-resnet50-last.h5'
# 是否載入已訓練模型
load_mode = True

In [5]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  channel_shift_range=10,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

In [6]:
# 將圖片資料集載入，並轉換成可訓練的數列資料（以flow_From_directory）
# 測試發現只需將圖片資料放在Python主檔同資料夾中即可（其中train在分出1/10的資料做驗證集）
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/training',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True, ) # set as training data

valid_batches = train_datagen.flow_from_directory(DATASET_PATH + '/validation',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True,) # set as validation data
# 這裡測試資料只做rescale
# 測試圖片資料集沒有預先分類，全放在test資料夾，而這裡有個特別的用法，如果用'./'test結果會是0 images 1 classes，必須用 '.'才能找到圖片

test_datagen = ImageDataGenerator(rescale=1./255)
test_batches = test_datagen.flow_from_directory(  '.',
                                                  classes = ['test'],
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode=None,
                                                  shuffle=False,
                                                  batch_size=1,)

Found 2543 images belonging to 5 classes.
Found 280 images belonging to 5 classes.
Found 2000 images belonging to 1 classes.


In [7]:
# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('\n')
for cls, idx in valid_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('\n')
for cls, idx in test_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


Class #0 = test


In [8]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    global load_mode
    
    if load_mode:
        lr = 1e-4
    else:
        lr = 1e-3
    if epoch > 150:
        lr *= 0.5e-3
    elif epoch > 120:
        lr *= 1e-3
    elif epoch > 90:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [9]:
# 準備保存模型的路徑
save_dir = os.path.join(DATASET_PATH,'save_models')
if not os.path.isdir(save_dir): #如果此路徑資料夾存在,則回傳True; 如不存在, 則回傳False(在這裡是會滿足if not的條件執行下一行程式)
    os.makedirs(save_dir) #建立指定的路徑與資料夾

# 保存訓練過程中效果最好的模型(避免訓練中斷時,浪費時間再重新訓練一次)
# 
checkpoint = ModelCheckpoint(filepath = save_dir,
                             monitor = 'val_accuracy', # 以此分數判斷模型效果有沒有變好
                             verbose = 1,
                             save_best_only = True # 只存最好的模型, 如果設False, 則每個比上一個好的模型都會被存下
                            )
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1),
                               cooldown = 0,
                               patience = 5,
                               min_r = 0.5e-6)
callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [10]:
# # 以預訓練模型ResNet50 為基礎來建立模型(遷移學習)
# if load_mode:
#     from tensorflow.keras.models import load_model
#     # 載入最後最好的模型
#     model_final = load_model(os.path.join(save_dir, WEIGHTS_FINAL))
# else:
    # 捨棄 ResNet50 頂層的 fully connected layers
model = ResNet50(include_top=True, weights='imagenet', input_tensor=None,
                input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
# remove classification layer
model.layers.pop()
model = Model(inputs=model.input, outputs=model.layers[-1].output)
x = model.output
# 增加Dropout層
x = Dropout(0.5)(x)
    
# 增加Dense Layer , 設定此模型5種分類的output , 並以 softmax產生各類別的機率值
output_layer = Dense(NUM_CLASSES, activation = 'softmax', name = 'softmax')(x)
    
model_final = Model(inputs=model.input, outputs=output_layer)
    
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
model_final.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

# 輸出整個網路結構
print(model_final.summary())

Learning rate:  0.0001
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                     

C:\Users\User\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# 訓練模型
model_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        verbose=1,
                        callbacks=callbacks)

# 儲存訓練好的模型
modelpath = os.path.join(save_dir, WEIGHTS_LAST)
model_final.save(modelpath)

<ipython-input-11-57f46f1e78e6>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_batches,


Learning rate:  0.0001
Epoch 1/200


In [ ]:
from keras.models import load_model

# identical to the best one
model_final = load_model(os.path.join(save_dir, WEIGHTS_FINAL))

test_batches.reset()
pred = model_final.predict_generator(test_batches, verbose=1)

In [ ]:
predicted_class_indices = np.argmax(pred, axis=1)
labels = (train_batches.class_indices)
label = dict((v,k) for k,v in labels.items())
predictions = [label[i] for i in predicted_class_indices]
# 修正檔名
filenames = test_batches.filenames
id = [name.replace('.jpg','').replace('all/','') for name in filenames]
# 輸出 csv 檔 id, flower_class
df = pd.DataFrame({"id":id, "flower_class":predicted_class_indices})
df.to_csv(DATASET_PATH + '/submission.csv', index=False)

In [ ]:
model_final.evaluate_generator(valid_batches, verbose=1)

In [1]:
!pip list

Package                            Version
---------------------------------- -------------------
-                                  nsorflow-gpu
-ensorflow-gpu                     2.7.0
absl-py                            1.0.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.0
backports.functools-lru-cache    